# Practical Error Mitigation On A Real Quantum Computer


This post appeared first on [Medium](https://towardsdatascience.com/practical-error-mitigation-on-a-real-quantum-computer-41a99dddf740) and my weekly [email-course](https://pyqml.substack.com/p/practical-error-mitigation-on-a-real?s=w).

Quantum computers are inherently unreliable. They are prone to errors that ruin any meaningful computation. Since our current devices are too small to correct the errors, today, the best we can do is to reduce their effect.

Unsurprisingly, research into quantum error mitigation is a major concern. A recent and promising error mitigation method is the Clifford Data Regression (CDR). It has been presented in [P. Czarnik et al., Error mitigation with Clifford quantum-circuit data, Quantum 5, 592 (2021)](https://arxiv.org/abs/2005.10189). In this method, we create a machine learning model that we can use to predict and mitigate the noise by using the data from quantum circuits that we can simulate classically.

In a previous post, we already  looked at (CDR). Furthermore, in another post, we used an open-source library, Mitiq, to apply this method on an exemplary quantum circuit. For this purpose, we added noise to our quantum simulator. Yet, the noise was pretty general, so it is hard to draw any conclusions on the effectiveness of this method.

Therefore, in this post, we adapt our code to work with a real quantum device.

We start with connecting to our IBMQ account. In the following listing, replace `TOKEN` with your actual API token. If you don't have one yet, [this post](https://towardsdatascience.com/how-to-run-code-on-a-real-quantum-computer-c1fc61ff5b4) explains how to get started with IBM Quantum and how to get the API token.

In [1]:
# load IBMQ Account data
from qiskit import IBMQ

# replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
provider = IBMQ.load_account()

Running code on an actual quantum computer is a time-consuming endeavor. IBM grants public access to their devices, so we're not the only to run our code. Once you sent a circuit, it will remain queued for quite some time before being executed.

So, let's insert an intermediate step and create a more realistic quantum simulation. For this purpose, we don't create a custom noise model but we obtain it from a real quantum device.

If you follow my weekly posts, you'll know that we're working our way towards participating in IBM's [Quantum Open Science Prize](https://research.ibm.com/blog/quantum-open-science-prize). They want us to simulate a Heisenberg model Hamiltonian for a three-particle system using Trotterization on their 7-qubit Jakarta device. Therefore, I will use this device today. If you haven't registered for this challenge, you don't have access to this device. In that case, I suggest to use the Quito system that we used in [this post](https://towardsdatascience.com/how-to-run-code-on-a-real-quantum-computer-c1fc61ff5b4), too.

The following listing shows how to load a noise model from an existing hardware.

In [2]:
from qiskit.providers.aer.noise import NoiseModel

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')
jakarta_noise_model = NoiseModel.from_backend(jakarta)


First, we need to get a provider (you need to use different parameter values to connect to `ibmq_quito`) that allows us to get access to a backend.

In this intermediate step, we use the backend as input to `NoiseModel.from_backend`. The result is a valid Qiskit noise model that we can feed into a quantum simulator.

In our case, we feed the noise model into the `execute_with_shots_and_noise` function that we imported from `mitiq`. We create a function (`sim_jakarta`) that takes a quantum circuit and produces the resulting expectation value. We use the same observable as in the [previous post]().

In [4]:
from mitiq.interface.mitiq_qiskit import qiskit_utils
from mitiq import Observable, PauliString

obs = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))

def sim_jakarta(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obs.matrix(), jakarta_noise_model, 4096)

We can use this function directly to obtain the expectation value of our observable. Notice that the resulting value is subject to noise. We see that it differs significantly from the ideal measurement.

In [7]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
print("ideal_measurement = ",ideal_measurement)

jakarta_unmitigated_measurement = obs.expectation(get_circuit(), sim_jakarta).real
print("unmitigated_measurement = ", jakarta_unmitigated_measurement)

ideal_measurement =  1.0599428216452071
unmitigated_measurement =  0.8542528961820239


In the above code, we use two more functions we introduced in the [previous post](). For completeness, these are `get_circuit` that creates the quantum circuit and the noiseless simulator `sim`.

In [6]:
from qiskit import QuantumCircuit

def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(pi/2, 0) # Clifford
        qc.rx(pi/2, 1) # Clifford
        
    return qc


def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)

We're now ready to run the full CDR with our simulated Jakarta backend. The only difference to the previous application is that we use the `sim_jakarta` function as the noisy executor parameter.

In [9]:
from mitiq import cdr

jakarta_mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    sim_jakarta,
    observable=obs.matrix(),
    simulator=sim,
    seed=0,
).real
print("mitigated_measurement = ", jakarta_mitigated_measurement)

mitigated_measurement =  1.0895031224814202


The comparison of the error-mitigated result to the unmitigated shows an error reduction of almost 90%. This is comparable to the improvement we saw with the general noise model.

In [10]:
jakarta_error_unmitigated = abs(jakarta_unmitigated_measurement-ideal_measurement)
jakarta_error_mitigated = abs(jakarta_mitigated_measurement-ideal_measurement)

print("Error (unmitigated):", jakarta_error_unmitigated)
print("Error (mitigated with CDR):", jakarta_error_mitigated)

print("Relative error (unmitigated):", (jakarta_error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):", jakarta_error_mitigated/ideal_measurement)

print(f"Error reduction with CDR: {(jakarta_error_unmitigated-jakarta_error_mitigated)/jakarta_error_unmitigated :.1%}.")

Error (unmitigated): 0.2056899254631832
Error (mitigated with CDR): 0.029560300836213083
Relative error (unmitigated): 0.19405756731662027
Relative error (mitigated with CDR): 0.027888580622047698
Error reduction with CDR: 85.6%.


Without further ado, let's now get our hands dirty with the real quantum device. The question is how to integrate the call to the real backend. Sending a quantum circuit to a real device is fairly easy. Let's check whether it works, anyway.

In [11]:
from qiskit import QuantumCircuit, transpile, assemble

# create a quantum circuit
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.measure_all()

mapped_circuit = transpile(circuit, backend=jakarta)
qobj = assemble(mapped_circuit, backend=jakarta, shots=1024)

# execute the circuit
job = jakarta.run(qobj)

# this may take a (long) while
result = job.result()

/tmp/ipykernel_59/3325198192.py:12: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  job = jakarta.run(qobj)


We create a simple quantum circuit that consists of a single qubit and a single Hadamard gate. The difference to working with a simulator is the following two steps. We need to transpile and assemble the circuit before we can feed it into the backend's `run` method. Once you ran this code, it's a good idea to log in to the [IBM Quantum console](https://quantum-computing.ibm.com/). 

There, open the "jobs"-page where you can see all your jobs and their status. In my case, I have to wait for about two hours for the job to run.

![IBMQ](./assets/ibmq.png)

Next, we need to write an executor that is not a simulator but one that connects to the real quantum device. Thus far, we used the `execute_with_noise_and_shots` function that we imported from Mitiq's Qiskit interface. Let's look at this function to see how it runs the circuit. You can find the source code in the [public repository](https://github.com/unitaryfund/mitiq/blob/master/mitiq/interface/mitiq_qiskit/qiskit_utils.py).

In line 166, you see how it runs the quantum circuit.

```python
# execution of the experiment
job = qiskit.execute(
    circ,
    backend=qiskit.Aer.get_backend("aer_simulator"),
    backend_options={"method": "density_matrix"},
    noise_model=noise_model,
    # we want all gates to be actually applied,
    # so we skip any circuit optimization
    basis_gates=basis_gates,
    optimization_level=0,
    shots=shots,
    seed_simulator=seed,
)
counts = job.result().get_counts()
```

They use Qiskit's execute function and get back a `job` object. This is the same kind of object we get back when running the circuit on a real device, [see my previous post](https://towardsdatascience.com/how-to-run-code-on-a-real-quantum-computer-c1fc61ff5b4).

So, let's rewrite this function to not run the local Qiskit `execute` but to transpile, assemble, and run the circuit on the real quantum device backend.

In [54]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, transpile, assemble

def execute_jakarta(
    circuit: QuantumCircuit,
    obs: np.ndarray,
    shots: int
) -> float:
    """Runs the circuit on the Jakarta device and returns
    the statistical estimate of the expectation value of the observable.
    Args:
        circuit: The input Qiskit circuit.
        obs: The observable to measure as a NumPy array.
        shots: The number of measurements.
    Returns:
        The expectation value of obs as a float.
    """
    # Avoid mutating circuit
    circ = circuit.copy()
    # we need to modify the circuit to measure obs in its eigenbasis
    # we do this by appending a unitary operation
    # obtains a U s.t. obs = U diag(eigvals) U^dag
    eigvals, U = np.linalg.eigh(obs)
    circ.unitary(np.linalg.inv(U), qubits=range(circ.num_qubits))

    circ.measure_all()


    # execution of the experiment
    """
    job = qiskit.execute(
        circ,
        backend=qiskit.Aer.get_backend("aer_simulator"),
        backend_options={"method": "density_matrix"},
        noise_model=noise_model,
        # we want all gates to be actually applied,
        # so we skip any circuit optimization
        optimization_level=0,
        shots=shots
    )
    """
    
    mapped_circuit = transpile(circ, backend=jakarta)
    qobj = assemble(mapped_circuit, backend=jakarta, shots=shots)

    # execute the circuit
    job = jakarta.run(qobj)
    
    counts = job.result().get_counts()
    expectation = 0

    for bitstring, count in counts.items():
        expectation += (
            eigvals[int(bitstring[0 : circ.num_qubits], 2)] * count / shots
        )
    return expectation


def real_jakarta(qc):
    return execute_jakarta(qc, obs.matrix(), 4096)

We removed the parameters `noise` and `seed` because we don't work with a simulation anymore. Further, we replaced the `qiskit.execute` call with the aforementioned procedure. The rest of the function remains unchanged.

Furthermore, we write another function `real_jakarta` that takes the quantum circuit and returns the calculated expectation value. It works similarly to the previously used simulators. So, let's use this to run the circuit and calculate the unmitigated measurement.

In [ ]:
real_jakarta_unmitigated_measurement = obs.expectation(get_circuit(), real_jakarta).real
print("real unmitigated_measurement = ", real_jakarta_unmitigated_measurement)

/tmp/ipykernel_86/4088215256.py:51: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  job = jakarta.run(qobj)


real unmitigated_measurement =  0.17124424416552098


Finally, we can run the CDR using the real Jakarta backend. When you start it, consider that the CDR runs several circuits to train the model. If you need to wait two hours per execution, it will take a whole day to compute all circuits.

In [ ]:
real_jakarta_mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    real_jakarta,
    observable=obs.matrix(),
    simulator=sim,
    seed=0,
).real
print("mitigated_measurement = ", real_jakarta_mitigated_measurement)

So, after all this waiting time, the results show that CDR mitigates almost 90% of the errors that result from the noise. These results are comparable to the results of the simulation.

The adaption of the Mitiq code to work with the real Jakarta device was pretty straightforward. 